### Import libraries

In [3]:
# Import libraries
import torch
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch dataset
from torch.utils.data.sampler import SubsetRandomSampler

# PyTorch model
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

### Load Dataset

In [4]:
df = open('./Training Dataset/play_style_train.csv').read().splitlines()
# df = df[:25000]

games = [i.split(',', 2)[-1] for i in df]
game_styles = [int(i.split(',', 2)[-2]) for i in df]

In [5]:
print(games[0])
print(game_styles[:10])

B[dq],W[pd],B[qp],W[dd],B[cf],W[do],B[ck]
[1, 1, 1, 1, 1, 2, 2, 2, 3, 3]


### Data processing

In [ ]:
# Set up coordinate
chars = 'abcdefghijklmnopqrs'
coordinates = {k: v for v, k in enumerate(chars)}

In [ ]:
def convert_game(moves):
    x = np.zeros((4, 19, 19))
    last_move_color = moves[-1][0]

    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        x[0, row, column] = 1

        if (color == last_move_color):
            x[2, row, column] = 1
        else:
            x[1, row, column] = 1

    last_move_column = coordinates[moves[-1][2]]
    last_move_row = coordinates[moves[-1][3]]
    x[3, row, column] = 1

    return x

In [ ]:
# Convert y into an one-hot array
def one_hot_encoding(value):
    one_hot = torch.eye(3)[value]
    one_hot = np.array(one_hot)
    return one_hot

In [7]:
x = []
for game in games:
    moves_list = game.split(',')
    x.append(convert_game(moves_list))
x = np.array(x)

Y = []
y = np.array(game_styles) - 1
for yi in y:
    Y.append(one_hot_encoding(yi))


### Data Transform

In [9]:
# flip and rotate the image to increase the training data
def flip_rotate(feature: np.ndarray, target: np.ndarray):
    a0 = np.array(feature)
    a1 = np.rot90(a0, axes=(2, 3))
    a2 = np.rot90(a1, axes=(2, 3))
    a3 = np.rot90(a2, axes=(2, 3))
    b0 = np.flip(a0, axis=3)
    b1 = np.rot90(b0, axes=(2, 3))
    b2 = np.rot90(b1, axes=(2, 3))
    b3 = np.rot90(b2, axes=(2, 3))

    c = np.concatenate((a0, a1, a2, a3, b0, b1, b2, b3), axis=0)

    d = np.concatenate((target, target, target, target,
                       target, target, target, target), axis=0)
    return c, d


In [10]:
x,y = flip_rotate(x, Y)

### Load Data

In [11]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 64
# percentage of training set to use as validation
valid_size = 0.1

In [12]:
# obtain training indices that will be used for validation
num_batch = len(x) // batch_size
num_train = num_batch * batch_size
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_batch)) * batch_size
train_idx, valid_idx = indices[split:], indices[:split]
print(len(x), batch_size, num_batch, num_train, split)

212920 64 3326 212864 21248


In [13]:
# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
print(len(train_sampler), len(valid_sampler))

191616 21248


In [14]:
# prepare data loaders (combine dataset and sampler)
x = torch.Tensor(x)
y = torch.Tensor(y)
# y = torch.stack(Y)

dataset = torch.utils.data.TensorDataset(x, y)

train_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_size,
    sampler=train_sampler,
    num_workers=num_workers
)
valid_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_size,
    sampler=valid_sampler,
    num_workers=num_workers
)

### Defind Model

In [15]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(4, 8, 7, padding=3)
        self.conv2 = nn.Conv2d(8, 16, 5, padding=2)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv4 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv5 = nn.Conv2d(64, 32, 3, padding=1)
        
        self.avgpool = nn.AvgPool2d(2, 2)

        self.fc1 = nn.Linear(32 * 9 * 9, 1024)
        self.fc2 = nn.Linear(1024 , 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 3)

        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.avgpool(F.relu(self.conv4(x)))
        x = F.relu(self.conv5(x))

        x = x.view(x.shape[0], -1) # flatten

        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = F.softmax(self.fc4(x), dim=1)
        
        return x

In [16]:
model = Network()

model

Network(
  (conv5): Conv2d(4, 8, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv1): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc4): Linear(in_features=2592, out_features=1024, bias=True)
  (fc1): Linear(in_features=1024, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=3, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, min_lr=5e-7,verbose=True)

### Training The Model

In [ ]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
    mps_device = torch.device("mps")
else:
    print('CUDA is available!  Training on GPU ...')
    
# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()
else:
    model.to(mps_device)

In [ ]:
epoch_num = 50

valid_loss_min = np.Inf

all_training_loss = []
all_validation_loss = []

for epoch in range(1, epoch_num+1):

    train_loss = 0.0
    valid_loss = 0.0
    
    class_correct = list(0. for i in range(3))
    class_total = list(0. for i in range(3))

    ##### train the model #####
    model.train()
    for data, target in train_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        else:
            data, target = data.to(mps_device), target.to(mps_device)
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        
        output = model(data)

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * data.size(0)  


    ##### validate the model #####
    model.eval()
    for data, target in valid_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        else:
            data, target = data.to(mps_device), target.to(mps_device)
       
        output = model(data) 
        loss = criterion(output, target)
        valid_loss += loss.item() * data.size(0)

        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)
        
        ans = []
        for i in range(batch_size):
            for j in range(3):
                if target[i][j]:
                    ans.append(j)
        ans = torch.tensor(np.array(ans))
        
        if train_on_gpu:
            pred = pred.cpu()
        else:
            pred = pred.cpu()

        correct_tensor = pred.eq(ans.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        
        # calculate test accuracy for each object class
        for i in range(batch_size):
            label = ans.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

    # calculate average losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)

    scheduler.step(valid_loss)

    all_training_loss.append(train_loss)
    all_validation_loss.append(valid_loss)    
    
    # print training/validation statistics 
    print('Epoch: {} \n\tTraining Loss: {:.6f} \n\tValidation Loss: {:.6f} [Minimum Validation Loss in the History: {:.6f}]'.format(
        epoch, train_loss, valid_loss, valid_loss_min))
    
    for i in range(3):
        if class_total[i] > 0:
            print('\t\tValidation Accuracy of style %d: %2d%% (%2d/%2d)' % (
                i + 1, 100 * class_correct[i] / class_total[i],
                np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('\t\tTest Accuracy of style: N/A (no training examples)' % (i + 1))
            
    print('\t\tTotal Accuracy: %2.2f%% (%2d/%2d)' % (100 * np.sum(class_correct) / np.sum(class_total), np.sum(class_correct), np.sum(class_total)))

    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('\t***** Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ... *****'.format(valid_loss_min, valid_loss))
        torch.save(model.state_dict(), 'model_1.pt')
        valid_loss_min = valid_loss
        
    
# showing the training loss and validation loss during the training process
plt.plot(np.array(all_training_loss))
plt.plot(np.array(all_validation_loss))
plt.show()

### Test the Trained Network

In [21]:
def Test(data_path: str, model, dict_path: str, output_path: str):

    # read file
    df_test = open(data_path).read().splitlines()

    # convert into formatted np array
    games = [i.split(',', 1)[-1] for i in df_test]
    game_ids = [i.split(',', 1)[0] for i in df_test]

    x_test = []
    for game in games:
        moves_list = game.split(',')
        filtered_list = [element for element in moves_list if element]
        x_test.append(convert_game(filtered_list))

    x_test = np.array(x_test)

    # convert the np array into torch tensor
    x_test = torch.Tensor(x_test)

    # load the dictionary
    model.load_state_dict(torch.load(
        dict_path, map_location=torch.device('cpu')))
    model.eval()

    # evaluate the result
    with torch.no_grad():
        output = model(x_test)

    # convert the result into value
    ans = []
    for o in output:
        if (o[0] > 0.5):
            ans.append(1)
        elif (o[1] > 0.5):
            ans.append(2)
        else:
            ans.append(3)

    # append the answer to the csv file
    with open(output_path, 'a+') as f:
        for i in range(len(ans)):
            f.write(f"{game_ids[i]},{ans[i]}\n")


In [22]:
# Set up coordinate
chars = 'abcdefghijklmnopqrs'
coordinates = {k: v for v, k in enumerate(chars)}

def convert_game(moves):
    x = np.zeros((4, 19, 19))
    last_move_color = moves[-1][0]

    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        x[0, row, column] = 1

        if (color == last_move_color):
            x[2, row, column] = 1
        else:
            x[1, row, column] = 1

    last_move_column = coordinates[moves[-1][2]]
    last_move_row = coordinates[moves[-1][3]]
    x[3, row, column] = 1

    return x

In [23]:
load_dict_name = './model_1.pt'
TEST_PATH = "./Testing Dataset/play_style_test_public.csv"
output_path = "./public_prediction_model_1.csv"

In [24]:
Test(TEST_PATH,model, load_dict_name,output_path)